# A locally adaptive normal Distribution

### Imports

In [0]:
colab = True

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')

    import os
    os.chdir('/content/gdrive/My Drive/02460AdvancedML/Code/')

    #!git clone https://github.com/rtqichen/torchdiffeq.git
    !pip install -e torchdiffeq
    
import torch
import numpy as np
from scipy.io import loadmat#'''##'''
from land.curve import *
from land.geodesics import *
from land.local_pca_metric import *
from IPython.display import Image
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import math
import sys
from time import perf_counter
import multiprocessing
from functools import partial
from contextlib import contextmanager

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Obtaining file:///content/gdrive/My%20Drive/02460AdvancedML/Code/torchdiffeq
  Found existing installation: torchdiffeq 0.0.1
    Can't uninstall 'torchdiffeq'. No files were found to uninstall.
  Running setup.py develop for torchdiffeq


In [0]:
def geodesic_minimizing_energy(curve, manifold, optimizer=torch.optim.LBFGS, max_iter=150, eval_grid=20):
    """
    Compute a geodesic curve connecting two points by minimizing its energy.

    Mandatory inputs:
    curve:      a curve object representing a curve with fixed end-points.
                When the function returns, this object has been updated to
                be a geodesic curve.
    manifold:   a manifold object representing the space over which the
                geodesic is defined. This object must provide a
                'curve_energy' function through which pytorch can
                back-propagate.
    
    Optional inputs:
    optimizer=torch.optim.LBFGS:    choice of iterative optimizer.
    max_iter=150:                   max number of iterations of the optimizer.
    eval_grid=20:                   number of points along the curve where
                                    energy is evaluated.
    """
    ## Initialize optimizer and set up closure
    alpha = torch.linspace(0, 1, eval_grid).reshape((-1, 1))
    opt = optimizer([curve.parameters])
    def closure():
        opt.zero_grad()
        loss = manifold.curve_energy(curve(alpha))
        loss.backward()
        return loss

    for iter in range(max_iter):
        opt.step(closure=closure)
        if torch.max(torch.abs(curve.parameters.grad)) < 1e-4:
            break

def shooting_geodesic(manifold, p, v, t=torch.linspace(0, 1, 50), requires_grad=False):
    if requires_grad:
        from torchdiffeq import odeint_adjoint as odeint
    else:
        from torchdiffeq import odeint
    odefunc = GeodesicODE(manifold)
    y = torch.cat([p.reshape(-1, 1), v.reshape(-1, 1)], dim=0) # (2D)xN
    retval = odeint(odefunc, y, t) # Tx(2D)xN
    D = retval.shape[1]//2
    c = retval[:, :D, :] # TxDxN
    dc = retval[:, D:, :] # TxDxN
    return c, dc

### Construct the metric

In [0]:
## Read data
data = torch.tensor(loadmat('data/mnist1000.mat')['data'], dtype=torch.float)
N, D = data.shape  #torch.Size([1000, 2])

In [0]:
## Parameters for metric
sigma = 0.1 #0.1
rho = 0.1 #0.1

## Create metric
M = LocalVarMetric(data=data, sigma=sigma, rho=rho)

### Glossaray and Notations

|Symbol  $\qquad$ $\qquad$ |              Explenation  |
|-|-|
|$\mathcal{C}$| Normalization Constant        |
|$D$| Dimensionality of the Data|
|$\mathrm{Exp}_x (v)$ | takes a tangent vector  $\mathbf{v} \in \mathcal{T} _x \mathcal{M}$ such that the curve $\gamma (t) =\mathrm{Exp}_x (t \cdot v)$ is a geodesic originatiing at x with initial velocity $\mathbf{v}$ and length $||\mathbf{v}||$   |
|$\gamma$| metric tensor |
|$\gamma'$| tangent vector, derivaitve (velocity) of $\gamma$ |
|$\mathrm{Log}_x (y) $ | inverse mapping, gives the initial tangent vector at point x|
|$\mathbf{\mu} $| Mean of a Gaussian Distribution |
|$\mathcal{M}$| Riemannian manifold |
|$M$| Riemannian Metric|
|$p_{\mathcal{M}} (\mathbf{x}|\mu, \mathbf{\Sigma} )$| Riemannian Normal distribution|
|$\phi$| 'data fitting term'|
|$\rho$| regularization parameter to avoid singular covariances|
|$\mathcal{S}_{++}^D$| Set of symmetric $D \times D$ positive definite matrices|
|$\mathbf{\Sigma}$ | Covariance Matrix |
|$\nabla$| Gradient of $\mu$|
|$\mathcal{T} _x \mathcal{M}$| tangent space at $x \in \mathcal{M}$|
|$\mathbf{v}$| Vector at a point in direction of geodesic to another point|
|$\mathbf{x}$ | random vector $\mathbf{x} \in \mathbb{R}^D $ (e.g. a point in the data)
|$x_{nd}$| $d^{th}$ dimension of the $n^{th} $ observation|

### Calculating $\mathrm{Exp}_p(V)$ 

Calculate $Exp_p(v)$ where p is the starting point and v is the velocity

use the function shooting-geodesic to calculate the necessary points
take last datapoint of the estimated curve

$q = Exp_p(v)$ where q is the end point

$\gamma' = dc = derivative of \gamma = velocity-on-each-point-in-c$

In [0]:
def ExpMap(p,v):
    c = shooting_geodesic(M,p,v)[0] 
    return c[-1].reshape(1, D)

### Calculating $\mathrm{Log}_p(q)$


Calculate $\mathrm{Log}_p(q)$ where p is the starting point and q is the end point

use the function geodesic_minimize_energy to calculate the initial vector at the starting point

$v = Log_x(y)$

In [0]:
def LogMap(p,q, num_nodes=10):
    C = CubicSpline(begin=p, end=q, num_nodes=num_nodes, requires_grad=True)
    geodesic_minimizing_energy(C, M)
    t = torch.linspace(0, 1, 2, dtype=C.begin.dtype)
    return C.deriv(t)[0].detach(),C

### Estimating the Normalization Constant

$\mathcal{C}(\mu,\Sigma)\simeq \frac{\mathcal{Z}}{S}\displaystyle\sum_{s=1}^{S} m(\mu,v_s), \quad $ where $ v_s\sim\mathcal(0,\Sigma)$

Normailzation constant of the Euclidean normal distribution:<br>
$\mathcal{Z}=\sqrt{(2\pi)^{D}|\Sigma|}$


m-function:

$m(\mu, \mathbf{v}) = \sqrt{|\mathrm{M}(\mathrm{Exp}_{\mu}(\mathbf{v}))|}$

---

**Algorithm 2** The estimation of the normalization constant $\mathcal{C}(\mu, \Sigma)$

---

**Input**: the given data $\{x_n\}_{n=1}^{N}$, the $\mu$, $\Sigma$, the number of samples $S$

**Output**: the estimated $\hat{C}(\mu, \Sigma)$

> **1:** sample $S$ tangent vectors $v_s\sim\mathcal(0,\Sigma)$ on $\mathcal{T}_\mu \mathcal{M}$

>  **2:**  map the $v_s$ on $\mathcal{M}$ as $\mathbf{x}_s =Exp_\mu(\mathbf{v}_s)$, $s=1,...,S$

> **3:** compute the normalization constant $\hat{C}(\mu, \Sigma) = \frac{Z}{S} \sum_{s=1}^{S} \sqrt{|(\text{M}(x_s)|}$

In [0]:
# tried both threading and multiprocessing -> mulit-processing is much faster
cpus = multiprocessing.cpu_count()

@contextmanager
def poolcontext(*args, **kwargs):
    pool = multiprocessing.Pool(*args, **kwargs)
    yield pool
    pool.terminate()

def calc_m(mu, v):
    # M is called to evaluate the metric --> gives the diagonal entries of the metric
    # The determinant of a diagonal matrix is the product of the values along the diagonal 
    # take mu as starting point, vector v as dirction, see where we end up, evaluate the curvature(metric) at the endpoint
    p = ExpMap(mu, v) 
    return M.metric(p).prod().sqrt() #returns scalar
    
def estimate_C(mu, Sigma, S=1000, v_s=None):
    #print("Estimating for S: "+str(S))
    # 1.: random samples from a Euclidean normal distribution with zero mean and covariance matrix Sigma
    if v_s is None:
        mvn = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.tensor([[0.0,0.0]]), covariance_matrix=Sigma)
        v_s = [mvn.sample().reshape(D,) for s in range(S)] #vectors in pytorch are 1-D and have shape: (D)

    #2./3.: Compute C using m(mu,v)
    # parallelisation
    with poolcontext(processes=cpus) as pool:
        m_s = pool.map(partial(calc_m, mu), v_s)

    Z = math.sqrt(((2*math.pi)**D)*torch.det(Sigma))

    C_hat = Z/S * sum(m_s)

    return C_hat

###  Eq. 12

### Objective Function (Eq. 11)

In [0]:
def calc_objective(mu, Sigma, C=None, datasamples=1000, S=1000):
    
    LL=0
    for n in random.sample(range(0,N),datasamples):
        L1 = LogMap(mu,data[n])[0]
        L2 = torch.mv(torch.inverse(Sigma), L1)
        LL += torch.dot(L1,L2).detach() 

    if C is None:
        C = estimate_C(mu, Sigma, S)
           
    phi = (LL/(2*datasamples)) + math.log(C)
  
    return phi

###  Equations for mu

Equation 12: $\nabla_{\mu} \phi(\mu,\Sigma)=-\Sigma^{-1}\left[\frac{1}{N} \sum_{n=1}^{N} Log_{\mu}\left(\mathbf{x}_{n}\right)-\frac{\mathcal{Z}}{\mathcal{C}(\mu, \Sigma) \cdot S} \sum_{s=1}^{S} m\left(\mu, \mathbf{v}_{s}\right) \mathbf{v}_{s}\right]$

In [0]:
def mu_step(mu, Sigma, S, v_s, C, alpha=0.01, data_samples=50):
    '''
    calculates the gradient for mu and takes one step in this direction
    '''
    
    Z = math.sqrt(((2*math.pi)**D)*torch.det(Sigma))

    term1 = (1 / data_samples) * torch.sum(torch.stack([LogMap(mu, data[n])[0] for n in random.sample(range(0, N), data_samples)]), dim=0)
    term2 = Z / (C*S) * torch.sum(torch.stack([calc_m(mu, v)*v for v in v_s]), dim = 0)

    nabla = term1-term2

    mu_p1 = ExpMap(mu, alpha*nabla)

    return mu_p1
    
    
    # --- notes ----
    #omit denominator because we do steepest descent instead of gradient descent. The covariance matrix is poorly estimated and mu is heavily reliant ont it. 
    #sgmnv = - torch.inverse(torch.tensor(Sigma,dtype=torch.float))
    #nabla = torch.mv(sgmnv, term1-term2)
    #---------------

In [0]:
def init_mu_step(mu, alpha=0.02, data_samples=50):
    '''
    Calculates the gradient for mu with intrinsic least squares and takes a step in this direction
    This is just the same as the first term in Eq. 12
    '''
    nabla = (1 / data_samples) * torch.sum(torch.stack([LogMap(mu, data[n])[0] for n in random.sample(range(0, N), data_samples)]), dim=0)
    mu = ExpMap(mu, alpha*nabla)
  
    return mu

### Equations for Sigma

Equation 13:  $\nabla_{\mathbf{A}} \phi(\boldsymbol{\mu}, \mathbf{\Sigma})=\mathbf{A}\left[\frac{1}{N} \sum_{n=1}^{N} \log _{\boldsymbol{\mu}}\left(\mathbf{x}_{n}\right) \log _{\boldsymbol{\mu}}\left(\mathbf{x}_{n}\right)^{\mathrm{T}}-\frac{\mathcal{Z}}{\mathcal{C}(\boldsymbol{\mu}, \boldsymbol{\Sigma}) \cdot S} \sum_{s=1}^{S} m\left(\boldsymbol{\mu}, \mathbf{v}_{s}\right) \mathbf{v}_{s} \mathbf{v}_{s}^{\top}\right]$

In [0]:
def Sigma_step(mu, A, Sigma, S, v_s, C, alpha=0.01, data_samples=50):
    '''
    calculates the gradient for Sigma and takes one step in this direction
    '''
    
    # equation (12) ------------------------------------------------------------
    # --- term1 ---
    loglist =[]
    for n in random.sample(range(0,N),data_samples):
        v = LogMap(mu,data[n])[0]   
        loglist.append(torch.ger(v, v))
    term1 = (1/data_samples)*torch.sum(torch.stack(loglist),dim=0) #2x2
    
    # --- term2 ---
    Z = math.sqrt(((2*math.pi)**D)*torch.det(Sigma)) # calculate Z w.r.t. Sigma
    term2 = (Z / (C*S)) * torch.sum(torch.stack([calc_m(mu, v)*torch.ger(v,v) for v in v_s]), dim = 0) #2x2
    
    # compute gradient of A (line 7)
    grad = torch.mm(A, term1-term2)
   
    # compute new A (line 8) ---------------------------------------------------                       
    A_p1 = A - (alpha * grad)     
     
    # compute new Sigma (line 9) -----------------------------------------------
    Sigma_p1 = torch.inverse(torch.mm(A_p1.transpose(0,-1), A_p1))
    
    return A_p1,Sigma_p1

In [0]:
def estimate_Sigma(mu):
    '''
    Estimate Sigma w.r.t. mu using intrinsic least squares 
    '''
   
    mu = torch.tensor([[0.1518, 0.3105]]) #estimated mu
    logs_tensor = torch.empty(N, 2) 

    for n in range(N):
        logs_tensor[n] = LogMap(mu, data[n])[0]

    Sigma_estimate = sum([torch.ger(logs,logs) for logs in logs_tensor])/(N-1)
    
    return Sigma_estimate

In [0]:
def calc_A(Sigma):
    
    #torch.set_printoptions(precision=10)
    
    '''
    Calculates the A for a given Sigma via eigen decomposition
    '''
   
    eig_vals, eig_vecs = torch.symeig(Sigma, eigenvectors=True)
    
    D_ = torch.tensor([[eig_vals[0], 0.0],[0.0, eig_vals[1]]])
    D_half_inv = torch.inverse(D_**0.5)

    A_t = torch.mm(eig_vecs, D_half_inv)
    A = torch.t(A_t)
    
    #check
    if not torch.all(torch.lt(torch.abs(torch.add(torch.inverse(Sigma), -torch.mm(A_t, A))), 1e-4)):
        
        print(torch.inverse(Sigma))
        print(torch.mm(A_t, A))
        
        
        raise ValueError('Sigma inverse does not merch A_t A')
        
    return A

In [0]:
stop here

SyntaxError: ignored

## Optimization runs

##### Initializing mu

In [0]:
#hyperparamters
alpha = 0.01
data_samples = 50
iters = 200
verbose = 1
ctr = 1

# start with mu as arithmetic mean
mu = torch.mean(data, dim=0).reshape(1, 2)

mu_init = torch.zeros(iters, D)
mu_init[0] = mu
    
while 1:
    if ctr >= 0: #replace with stop condition
        break
        
    if ctr > 150:
        alpha = 0.005
        
    mu_p1 = init_mu_step(mu, alpha, data_samples)
    
    #for saving
    mu_init[ctr]= mu_p1
    
    if not ctr % verbose:
        print("{}:".format(ctr))
        print("mu:")
        print(mu_p1)
        
        torch.save(mu_init, 'results_MNIST/mu_init.pt')
        
    #update parameters
    mu = mu_p1
    ctr += 1

##### Fitting only mu with Constant Sigma

In [0]:
#hyperparamters
alpha = 0.01
S = 500
data_samples = 50
iters = 50
ctr = 1
verbose = 1

# estimations via intrinsic least squares
mu = torch.tensor([[0.1491, 0.3293]]) 
Sigma = torch.tensor([[ 1.2289, -0.0078],
        [-0.0078,  0.0124]])

#for saving
mu_opti = torch.zeros(iters, D)
C_mu_opti = torch.zeros(iters)
mu_opti[0]=mu

while 1:
    if ctr>=0:#replace with stop condition or iters
        break
    if ctr>=0.9*iters:
        alpha = 0.005
        data_samples = 1000
  
    # random sample vectors
    mvn = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.tensor([[0.0,0.0]]), covariance_matrix=Sigma)
    v_s = [mvn.sample().reshape(D,) for _ in range(S)]#we use 1-D vectors
    
    # estimate C
    C = estimate_C(mu, Sigma, S, v_s)
    
    # take Step
    mu_p1 = mu_step(mu, Sigma, S, v_s, C, alpha, data_samples)
      
    #for saving
    mu_opti[ctr] = mu_p1
    C_mu_opti[ctr-1] = C
    
    if not ctr % verbose:
        print(mu_p1)
        torch.save(mu_opti, 'results_MNIST/mu_opti.pt')
        torch.save(C_mu_opti, 'results_MNIST/C_mu_opti.pt')

    #update parameters
    mu = mu_p1
    ctr += 1  

#save all data
#C_mu_opti[ctr-1] = estimate_C(mu, Sigma, S, v_s)
#torch.save(mu_opti, 'results_MNIST/mu_opti.pt')
#torch.save(C_mu_opti, 'results_MNIST/C_mu_opti.pt')

##### Fitting only Sigma with constant mu

In [0]:
#initial values
alpha = 0.01
S = 500
data_samples = 50
iters = 100
ctr = 1
verbose = 1

#optimized mu, estimated Sigma
mu = torch.tensor([[0.1635, 0.3242]])

#Sigma = estimate_Sigma(mu)
Sigma = torch.tensor([[ 1.2289, -0.0078],
        [-0.0078,  0.0124]])

A = calc_A(Sigma)

#for saving
Sigma_opti = torch.zeros(iters, D, D)
C_Sigma_opti = torch.zeros(iters)
Sigma_opti[0]=Sigma

# while loop for optimisation
while 1:
    if ctr > iters: #replace with stop condition or iters
        break
        
    if ctr>=0.9*iters:
        alpha = 0.005
        data_samples = 1000
    
    # random sample vectors
    mvn = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.tensor([[0.0,0.0]]), covariance_matrix=Sigma)
    v_s = [mvn.sample().reshape(D,) for _ in range(S)]   #we use 1-D vectors

    #estimate C
    C = estimate_C(mu, Sigma, S, v_s)
    
    #take Step
    A_p1,Sigma_p1 = Sigma_step(mu, A, Sigma, S, v_s, C, alpha, data_samples)
    
    #for saving
    Sigma_opti[ctr] = Sigma_p1
    C_Sigma_opti[ctr-1] = C
    
    if not ctr % verbose:
        print("{}:".format(ctr))
        print("Sigma:")
        print(Sigma_p1)
        print("A:")
        print(A_p1)
        
        torch.save(Sigma_opti, 'results_MNIST/Sigma_opti.pt')
        torch.save(C_Sigma_opti, 'results_MNIST/C_Sigma_opti.pt')
        
    #update parameters
    A,Sigma=A_p1,Sigma_p1
    ctr+=1    

#save all data
C_Sigma_opti[ctr-1] = estimate_C(mu, Sigma, S, v_s)
torch.save(Sigma_opti, 'results_MNIST/Sigma_opti')
torch.save(C_Sigma_opti, 'results_MNIST/C_Sigma_opti')    
    
# --------------------------------------------------------------------------
# Torch notes:
# easiest way to do vectors in pytorch is in 1-D -> shape (D) 
# other way are matrices, that are 2-D : shape(D,1) D rows, 1 column
# torch.ger(v,v) is just the same as v*v^T -> outer product of two vectors
# torch.mm - matrix multiplication (c_ij = a_i1*b_i1 + ... + a_im*b_im)
# A*B - elementwise muliplication of two matrices (Hadamard product)-- mathematical notation would be: A ∘ B

##### Combine mu and Sigma

In [0]:
# initialise values
alpha = 0.01
S=1000
data_samples = 50
iters = 100
ctr= 60
verbose = 1

#estimated mu, estimated Sigma, derived A
mu = torch.tensor([[0.1491, 0.3293]]) 

#Sigma = estimate_Sigma(mu)
Sigma = torch.tensor([[ 1.2289, -0.0078],
        [-0.0078,  0.0124]])

#after 59 steps
mu = torch.tensor([0.1452, 0.3245])
Sigma = torch.tensor([[ 0.5471, -0.0011],
        [-0.0011,  0.0113]])

print(Sigma)

A = calc_A(Sigma)

mu_merged  = torch.load('results_MNIST/mu_ad_steps.pt')
Sigma_merged = torch.load('results_MNIST/Sigma_ad_steps.pt')
C_merged = torch.load('results_MNIST/C_ad_steps.pt')
#mu_merged[0]=mu
#Sigma_merged[0]=Sigma


# while loop for optimisation
while 1:
    if ctr >= iters: #replace with stop condition
        break
        
    if ctr>0:
        alpha = 0.1
    
    if ctr>=20:
        alpha = 0.05    
    
    if ctr>=40:
        alpha = 0.025   
    
    if ctr>=60:
        alpha = 0.01
    
    if ctr>=80:
        aplpha = 0.005
        data_samples=N
    
    print("current alpha: {}, sample_size:{}".format(alpha, data_samples))
    
    
    t1 = perf_counter() # see how long we need
    
    
    # mu -----------------------------------------------------------------------
    # random sample vectors
    mvn = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.tensor([[0.0,0.0]]), covariance_matrix=Sigma)
    v_s = [mvn.sample().reshape(D,) for _ in range(S)]#we use 1-D vectors
    # estimate C
    C = estimate_C(mu, Sigma, S, v_s)
    # take Step
    mu_p1 = mu_step(mu, Sigma, S, v_s, C, alpha, data_samples)
    
    # Sigma --------------------------------------------------------------------
    # random sample vectors
    mvn = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.tensor([[0.0,0.0]]), covariance_matrix=Sigma)
    v_s = [mvn.sample().reshape(D,) for s in range(S)]#we use 1-D vectors
    #estimate C
    C = estimate_C(mu_p1, Sigma, S, v_s)
    #take Step
    A_p1,Sigma_p1 = Sigma_step(mu, A, Sigma, S, v_s, C, alpha, data_samples)
    
    
    #for saving
    #for saving
    mu_merged[ctr] = mu_p1
    Sigma_merged[ctr] = Sigma_p1
    C_merged[ctr-1] = C
    
    if not ctr % verbose:
        print("{}:".format(ctr))
        print("mu:")
        print(mu_p1)
        print("Sigma:")
        print(Sigma_p1)
        print("A:")
        print(A_p1)
        
        torch.save(mu_merged, 'results_MNIST/mu_ad_steps.pt')
        torch.save(Sigma_merged, 'results_MNIST/Sigma_ad_steps.pt')
        torch.save(C_merged, 'results_MNIST/C_ad_steps.pt')
        
        t2 = perf_counter()-t1
        print("current time per step: {:3.2f}".format(t2))
        
    #update parameters
    mu = mu_p1
    A = A_p1
    Sigma = Sigma_p1
    ctr+=1    

tensor([[ 0.5471, -0.0011],
        [-0.0011,  0.0113]])
current alpha: 0.01, sample_size:50
60:
mu:
tensor([[0.1431, 0.3246]])
Sigma:
tensor([[ 0.5494, -0.0012],
        [-0.0012,  0.0113]])
A:
tensor([[-2.0447e-02, -9.4088e+00],
        [-1.3491e+00,  2.6071e-03]])
current time per step: 373.57
current alpha: 0.01, sample_size:50
61:
mu:
tensor([[0.1410, 0.3248]])
Sigma:
tensor([[ 0.5505, -0.0011],
        [-0.0011,  0.0113]])
A:
tensor([[-1.9733e-02, -9.4100e+00],
        [-1.3477e+00,  2.7070e-03]])
current time per step: 383.11
current alpha: 0.01, sample_size:50
62:
mu:
tensor([[0.1406, 0.3249]])
Sigma:
tensor([[ 0.5509, -0.0011],
        [-0.0011,  0.0113]])
A:
tensor([[-1.9233e-02, -9.4111e+00],
        [-1.3473e+00,  2.7780e-03]])
current time per step: 388.64
current alpha: 0.01, sample_size:50
63:
mu:
tensor([[0.1372, 0.3248]])
Sigma:
tensor([[ 0.5479, -0.0012],
        [-0.0012,  0.0113]])
A:
tensor([[-2.0447e-02, -9.4129e+00],
        [-1.3510e+00,  2.6123e-03]])
current t

In [0]:
mu_merged= torch.load('results_MNIST/mu_ad_steps.pt')
C_merged[ctr-1] = estimate_C(mu_merged[-1].reshape(1,2), Sigma_merged[-1], S, v_s)
torch.save(C_merged, 'results_MNIST/C_ad_steps.pt')

In [0]:
torch.load('results_MNIST/Sigma_ad_steps.pt')


tensor([[[ 1.2289e+00, -7.8000e-03],
         [-7.8000e-03,  1.2400e-02]],

        [[ 8.2857e-01, -6.6626e-03],
         [-6.6626e-03,  1.2358e-02]],

        [[ 7.0677e-01, -3.6605e-03],
         [-3.6605e-03,  1.2277e-02]],

        [[ 6.3227e-01, -6.2028e-03],
         [-6.2028e-03,  1.2277e-02]],

        [[ 5.9995e-01, -7.9561e-03],
         [-7.9561e-03,  1.2293e-02]],

        [[ 5.6177e-01, -7.3107e-03],
         [-7.3107e-03,  1.2227e-02]],

        [[ 5.7630e-01, -6.2754e-03],
         [-6.2754e-03,  1.2143e-02]],

        [[ 5.6431e-01, -9.3543e-03],
         [-9.3543e-03,  1.2211e-02]],

        [[ 5.8292e-01, -1.2617e-02],
         [-1.2617e-02,  1.2285e-02]],

        [[ 5.9051e-01, -1.1690e-02],
         [-1.1690e-02,  1.2190e-02]],

        [[ 5.7469e-01, -1.0926e-02],
         [-1.0926e-02,  1.2184e-02]],

        [[ 5.5923e-01, -1.0447e-02],
         [-1.0447e-02,  1.2124e-02]],

        [[ 5.3653e-01, -9.1445e-03],
         [-9.1445e-03,  1.2149e-02]],

        [[ 5

In [0]:
mu = torch.load('results_MNIST/mu_merged.pt')
Sigma = torch.load('results_MNIST/Sigma_merged.pt')
C = torch.load('results_MNIST/C_merged.pt')

In [0]:
for i in range(100, len(mu)):
    print(i)
    phi.append(calc_objective(mu[i].reshape(1,2), Sigma[i], C[i], datasamples=1000, S=500))
    print(phi[i])
    torch.save(phi,'results_MNIST/phi_merged.pt')
    

In [0]:
# estimations via intrinsic least squares
mu = torch.tensor([[0.1491, 0.3293]]) 
Sigma = torch.tensor([[ 1.2289, -0.0078],
        [-0.0078,  0.0124]])

S_save=[]

for S in range(1,3002,100):
    print(S)
    S_save.append(estimate_C(mu, Sigma, S))
    torch.save(S_save, 'results_MNIST/S_3000.pt')

In [0]:
torch.load('results_MNIST/Sigma_ad_steps.pt')[59]

tensor([[ 0.5471, -0.0011],
        [-0.0011,  0.0113]])